## 3 rings Samwise target 10G new asymmetric threshold test

In [2]:
import numpy as np
import magpylib as magpy
from maggeometry import n_rings, four_rings
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_xy_grid

In [3]:
Br = 1.09e3
mag_dir = (0,0,1)
mirror1_z = False
ringset_1_config = [Br, mag_dir, mirror1_z]

r = 1.09e3
mag_dir = (0,0,1)
mirror2_z = True
ringset_2_config = [Br, mag_dir, mirror2_z]
# innerrad1, width1, thickness1, innerrad2, width2, thickness2, dist2
bounds = ((1, 150), (2, 20), (2, 20),
          (1, 150), (2, 20), (2, 20), (1, 300))

In [8]:
def get_grid_mag_and_target_nonuniformity(magnets, grid, grid_res, target=-10, use_z=True):
    mT_to_G = 10
    B = magnets.getB(grid)
    G = B * mT_to_G
    # find magnitude of the b-field at all points over grid
    Gmag = 0
    if use_z:
        Gmag = G[:,:,2]
    else:
        Gmag = np.linalg.norm(G, axis=2)
    # find magnitude of b-field at center of grid
    mid_id = int(grid_res/2)
    Gcenter = Gmag[mid_id][mid_id]
    # calculate nonuniformity with respect to center b-field
    Gnon = np.abs((Gmag - Gcenter)/Gcenter)
    Gtarget_non = np.abs((Gmag - target)/target)

    av_nonuniformity = np.sum(Gnon) / grid_res**2
    av_target_nonuniformity = np.sum(Gtarget_non) / grid_res**2
#     return Gmag, Gnon, Gcenter, av_nonuniformity, av_target_nonuniformity, max_nonuniformity
    return Gcenter, av_nonuniformity, av_target_nonuniformity

In [9]:
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
low_threshold = -0.1 # in G
high_threshold = 1e-6
target = -10
def obj1_nonuniformity(x):
    obj_ring_config=[[*ringset_1_config, *x[0:3], 0], [*ringset_2_config, *x[3:7]]]
    magnets = n_rings(obj_ring_config)
    center_field, av_nonuniformity, av_target_nonuniformity = get_grid_mag_and_target_nonuniformity(magnets, grid, grid_res, target)

    # triggered only when center_field > -10.1 or center_field < -10+1e-6
    if ((center_field - target < low_threshold) or (center_field - target > high_threshold)):
        return av_target_nonuniformity
    return av_nonuniformity

In [10]:
import numpy as np
from scipy.optimize import minimize, Bounds
from maggeometry import n_rings
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_grid
from alive_progress import alive_bar

options = {'disp': True, 'fatol': 1e-18, 'maxiter': 5e3, 'adaptive': True}
method = "Nelder-Mead"

shots = 5
results_x = []
results_fun = []
guesses_x0 = []
with alive_bar(shots, force_tty=True) as bar:
    for i in range(shots):
        x0 = np.zeros(len(bounds))
        for b in range(len(bounds)):
            x0[b] = np.random.uniform(low=bounds[b][0], high=bounds[b][1])
        res = minimize(obj1_nonuniformity, x0, method=method, options=options, bounds=bounds)
        guesses_x0.append(x0)
        results_x.append(res.x)
        print(res.x)
        results_fun.append(res.fun)
        bar()

print("Objective function values:")
print(results_fun)
print("Result parameters:")
print(results_x)
print("Starting guesses:")
print(guesses_x0)

on 0: [125.35694174   2.           6.21443315 148.34655845  16.08220446         
         2.89539756  78.54471838]


on 0: /var/folders/06/7pjfbw510cz2ch1m7lx8vvl80000gn/T/ipykernel_52612/3812064488.py:19: RuntimeWarning: Maximum number of iterations has been exceeded.
        res = minimize(obj1_nonuniformity, x0, method=method, options=options, bounds=bounds)


on 1: [150.           2.09485129  20.           1.00000001   2.                 
         2.         299.99999063]
on 2: [150.          20.           2.03674937 147.6675059    2.                 
        19.99050472  96.16439131]
on 3: [149.99998464   2.50723649  16.72245346   1.00013266   2.00002551         
         2.00016796 299.98282842]
on 4: [150.           2.09485129  20.           1.00000033   2.00000045         
         2.         299.99996346]
|████████████████████████████████████████| 5/5 [100%] in 26:06.1 (0.00/s)       
Objective function values:
[3.152755349175679e-08, 0.0034778349915031473, 2.331996704536812e-06, 0.0034808700549654446, 0.003477834993918656]
Result parameters:
[array([125.35694174,   2.        ,   6.21443315, 148.34655845,
        16.08220446,   2.89539756,  78.54471838]), array([150.        ,   2.09485129,  20.        ,   1.00000001,
         2.        ,   2.        , 299.99999063]), array([150.        ,  20.        ,   2.03674937, 147.6675059 ,
      

In [12]:
from maggeometry import n_rings
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_xy_grid
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
results_g_center = []
results_nonun = []
for x in results_x:
    obj_ring_config=[[*ringset_1_config, *x[0:3], 0], [*ringset_2_config, *x[3:7]]]
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    results_g_center.append(center_field)
    results_nonun.append(av_nonuniformity)
print(results_g_center)
print(results_nonun)

[10.074453543433801, 9.936704812516142, 10.09999068263915, 9.936648513284574, 9.936704812451527]
[3.152755349175679e-08, 0.0066774771653548655, 2.331996704536812e-06, 0.006683282554535431, 0.006677477170077799]


In [13]:
from maghelper import make_opt_res_csv
import datetime
today = datetime.datetime.today().strftime('%Y_%m_%d')
r_x = np.array(results_x)
r_x = np.insert(r_x, 3, 0, axis=1)
r_g = np.array(guesses_x0)
r_g = np.insert(r_g, 3, 0, axis=1)
make_opt_res_csv(f'results/3 rings/{today}_samwise_at_3r_1090mT_', results_fun, results_g_center, r_x, r_g)

Number of unique results: 5
   nonuniformity  center_field_gauss  r_1_innerrad  r_1_width  r_1_thickness  \
0   3.152755e-08           10.074454    125.356942   2.000000       6.214433   
1   2.331997e-06           10.099991    150.000000  20.000000       2.036749   
2   3.477835e-03            9.936705    150.000000   2.094851      20.000000   
3   3.477835e-03            9.936705    150.000000   2.094851      20.000000   
4   3.480870e-03            9.936649    149.999985   2.507236      16.722453   

   r_1_dist  r_2_innerrad  r_2_width  r_2_thickness    r_2_dist  g_1_innerrad  \
0       0.0    148.346558  16.082204       2.895398   78.544718     33.295941   
1       0.0    147.667506   2.000000      19.990505   96.164391     73.124998   
2       0.0      1.000000   2.000000       2.000000  299.999991     34.459104   
3       0.0      1.000000   2.000000       2.000000  299.999963     24.218414   
4       0.0      1.000133   2.000026       2.000168  299.982828     47.836851   

   g

(array([[3.15275535e-08, 1.00744535e+01, 1.25356942e+02, 2.00000000e+00,
         6.21443315e+00, 0.00000000e+00, 1.48346558e+02, 1.60822045e+01,
         2.89539756e+00, 7.85447184e+01, 3.32959414e+01, 3.45989641e+00,
         1.06706350e+01, 0.00000000e+00, 5.79969102e+01, 1.32371653e+01,
         1.49031069e+01, 2.57901522e+02],
        [2.33199670e-06, 1.00999907e+01, 1.50000000e+02, 2.00000000e+01,
         2.03674937e+00, 0.00000000e+00, 1.47667506e+02, 2.00000000e+00,
         1.99905047e+01, 9.61643913e+01, 7.31249978e+01, 1.83141563e+01,
         1.15732416e+01, 0.00000000e+00, 1.23748746e+02, 6.80042148e+00,
         8.81380941e+00, 2.95212937e+02],
        [3.47783499e-03, 9.93670481e+00, 1.50000000e+02, 2.09485129e+00,
         2.00000000e+01, 0.00000000e+00, 1.00000001e+00, 2.00000000e+00,
         2.00000000e+00, 2.99999991e+02, 3.44591045e+01, 2.37658227e+00,
         4.39002415e+00, 0.00000000e+00, 4.29056248e+01, 1.58029788e+01,
         3.85553777e+00, 9.27368727e+01]